[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanchess98/Notebooks-Deep-Learning/blob/transfer_learning/transfer_learning.ipynb)

# Transfer Learning (Enfoque por cuadrantes o escenarios usando Pytorch)

Aprender una tarea nueva puede ser un proceso que lleve una considerable cantidad de tiempo. Sin embargo, los seres humanos tienen la capacidad de asociar nuevas tareas con otras que ya sabe como ejecutar y aplicar este conocimiento para reducir el tiempo y costo de aprendizaje. Por ejemplo, si una persona sabe como manejar una bicicleta, para esa persona será mucho más fácil aprender a manejar una motocicleta. Asimismo, una persona que domine el idioma español tendrá mucha más facilidad para aprender otro idioma similar, como el portugués, que una persona de origen alemán o japonés.

Este mismo concepto se puede aplicar a las máquinas inteligentes y se le conoce como *Transfer Learning* o Aprendizaje por transferencia. En este caso, un modelo desarrollado para una resolver una tarea puede usarse como punto de partida para resolver otra tarea similar.

Pueden presentarse diferentes escenarios en la aplicación de *transferencia de aprendizaje* a un dataset nuevo, pero generalmente se puede reducir a cuatro casos:

1. La nueva base de datos es numerosa y se parece muy poco a la base de datos del modelo pre-entrenado
2. La nueva base de datos es numerosa y parecida a la base de datos del modelo pre-entrenado
3. La nueva base de datos es pequeña y diferente de la base de datos del modelo pre-entrenado
4. La nueva base de datos es pequeña y parecida a la base de datos del modelo pre-entrenado

En la siguiente gráfica tipo plano cartesiano pueden visualizarce mejor los posibles escenarios, dividiendo el plano en cuadrantes que representan cada escenario y dónde en la dirección de crecimiento del eje $x$ se indica la similaridad de los dos datasets y el eje $y$ el tamaño del nuevo dataset.

**(Insertar imagen de cuadrantes aquí)**

Para cada escenario existe una metodología diferente a implementar y en el curso de este notebook trataremos de ejemplificar cada una con la ayuda de los frameworks de Keras y Pytorch.

## Caso 1. La nueva base de datos es numerosa y diferente a la del modelo pre-entrenado

Si la nueva base de datos es grande y no se parece a la original, se recomienda seguir los siguientes pasos:

1. Quitar la última capa totalmente conectada y añadir una nueva capa totalmente conectada cuya dimensión de salida se igual al número de clases de la nueva base de datos.

2. Aleatorizar los pesos de la nueva capa totalmente conectada e inicializarlos con valores aleatorios

3. Entrenar la red para actualizar los pesos de la nueva capa totalmente conectada.


### Cargar base de datos

Comenzamos cargando la nueva base de datos que en este caso de trata de CIFAR10, Las cuales tienen simensión de 3x32x32, es decir, 3 canales y resolución 32x32 píxeles. Asimismo, cuenta con 10 clases que son:  ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Cargar el modelo preentrenado

In [1]:
from torchvision import datasets, models, transforms

In [2]:
pretrained_model = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/david/.cache/torch/checkpoints/vgg16-397923af.pth
100.0%
